In [1]:
%load_ext autotime

import os, math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm, trange

In [2]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Input, BatchNormalization, Bidirectional
from sklearn.metrics import mean_squared_error, mean_absolute_error

def model(test_data, train_data):
    test_time = test_data['time']
    test_gl_value = test_data['gl_value']
    
    test_data.drop(columns = ['time'], inplace = True)
    train_data.rename(columns = {'bolus_type_normal dual':'bolus_type_normal_dual', 'bolus_type_square dual':'bolus_type_square_dual'}, inplace = True)
    train_data.drop(columns = ['time'], inplace = True)
    
    empty_train_col = [0]*len(train_data)
    for i, item in enumerate(test_data.columns):
        if item not in train_data.columns:
            train_data.insert(i, item, empty_train_col)

    empty_test_col = [0]*len(test_data)
    for i, item in enumerate(train_data.columns):
        if item not in test_data.columns:
            test_data.insert(i, item, empty_test_col)
    
    X_data = train_data.drop(columns = ['gl_predict'])
    y_data = train_data[['gl_predict']]
    input_dim = X_data.shape[1]
    
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    scaler_x.fit(X_data)
    scaler_y.fit(y_data)
    scaled_X_train_data = scaler_x.transform(X_data)
    scaled_y_train_data = scaler_y.transform(y_data)
    scaled_X_train_data = np.reshape(scaled_X_train_data, (scaled_X_train_data.shape[0], 1, scaled_X_train_data.shape[1]))
    
    model = Sequential()
    model.add(Bidirectional(LSTM(128, input_shape = (scaled_X_train_data.shape[1], scaled_X_train_data.shape[2]))))
    model.add(Dense(150, activation = 'relu'))
    model.add(Dropout(0.20))
    model.add(Dense(100, activation = 'relu'))
    model.add(Dropout(0.15))
    model.add(Dense(50, activation = 'relu'))
    model.add(Dense(20, activation = 'relu'))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse', optimizer = 'adam')
    model.fit(scaled_X_train_data, scaled_y_train_data, epochs = 200, batch_size = 32, shuffle = False)
    
    X_test_data = test_data.drop(columns = ['gl_predict'])
    y_test_data = test_data[['gl_predict']]
    scaled_X_test_data = scaler_x.transform(X_test_data)
    scaled_X_test_data = np.reshape(scaled_X_test_data, (scaled_X_test_data.shape[0], 1, scaled_X_test_data.shape[1]))
    prediction = model.predict(scaled_X_test_data, batch_size = 32)
    scaled_prediction = scaler_y.inverse_transform(prediction)
    
    final_output = pd.DataFrame(columns = ['timestamp', 'predicted_BGL_value'])
    final_output['timestamp'] = test_time
#     final_output['true_BGL_value'] = test_gl_value
    final_output['predicted_BGL_value'] = scaled_prediction
    
    return (final_output, math.sqrt(mean_squared_error(scaled_prediction, y_test_data)), mean_absolute_error(scaled_prediction, y_test_data))

Using TensorFlow backend.


time: 2.41 s


In [3]:
test_data_list = ['540-ws-training.csv', '544-ws-training.csv', '552-ws-training.csv', '567-ws-training.csv', '584-ws-training.csv', '596-ws-training.csv']

rmse_list = {}
mae_list = {}
for file in os.listdir('../data/derived/30_min_complete_dataset/'):
    if file in test_data_list:
        train_data = pd.read_csv('../data/derived/30_min_complete_dataset/'+file)
        test_data = pd.read_csv('../data/derived_test/30_min_complete_dataset/'+file[0:7]+'testing.csv')
        final_output, rmse_error, mae_error = model(test_data, train_data)
        rmse_list[file] = rmse_error
        mae_list[file] = mae_error
        print ('{} - {}'.format(file, rmse_error))
        print ('{} - {}'.format(file, mae_error))
        final_output.to_csv('../data/output/bidirectional_lstm/'+file[0:7]+'result_30min.csv')

Epoch 1/200
2207/2207 [==============================] - 2s 1ms/step - loss: 0.0516
Epoch 2/200
2207/2207 [==============================] - 0s 219us/step - loss: 0.0128
Epoch 3/200
2207/2207 [==============================] - 0s 182us/step - loss: 0.0076
Epoch 4/200
2207/2207 [==============================] - 0s 185us/step - loss: 0.0071
Epoch 5/200
2207/2207 [==============================] - 0s 219us/step - loss: 0.0076 0s - loss:
Epoch 6/200
2207/2207 [==============================] - 0s 225us/step - loss: 0.0070
Epoch 7/200
2207/2207 [==============================] - 0s 192us/step - loss: 0.0067
Epoch 8/200
2207/2207 [==============================] - 0s 163us/step - loss: 0.0066
Epoch 9/200
2207/2207 [==============================] - 0s 172us/step - loss: 0.0065
Epoch 10/200
2207/2207 [==============================] - 0s 162us/step - loss: 0.0067
Epoch 11/200
2207/2207 [==============================] - 0s 197us/step - loss: 0.0064
Epoch 12/200
2207/2207 [===================

2207/2207 [==============================] - 0s 223us/step - loss: 0.0063
Epoch 95/200
2207/2207 [==============================] - 1s 262us/step - loss: 0.0060
Epoch 96/200
2207/2207 [==============================] - 1s 275us/step - loss: 0.0058
Epoch 97/200
2207/2207 [==============================] - 1s 252us/step - loss: 0.0058
Epoch 98/200
2207/2207 [==============================] - 1s 248us/step - loss: 0.0059
Epoch 99/200
2207/2207 [==============================] - 1s 249us/step - loss: 0.0057
Epoch 100/200
2207/2207 [==============================] - 1s 239us/step - loss: 0.0056
Epoch 101/200
2207/2207 [==============================] - 1s 263us/step - loss: 0.0056
Epoch 102/200
2207/2207 [==============================] - 1s 274us/step - loss: 0.0058
Epoch 103/200
2207/2207 [==============================] - 1s 278us/step - loss: 0.0056
Epoch 104/200
2207/2207 [==============================] - 1s 227us/step - loss: 0.0057
Epoch 105/200
2207/2207 [==========================

Epoch 187/200
2207/2207 [==============================] - 1s 249us/step - loss: 0.0051
Epoch 188/200
2207/2207 [==============================] - 1s 251us/step - loss: 0.0051
Epoch 189/200
2207/2207 [==============================] - 1s 256us/step - loss: 0.0052
Epoch 190/200
2207/2207 [==============================] - 1s 257us/step - loss: 0.0051
Epoch 191/200
2207/2207 [==============================] - 1s 256us/step - loss: 0.0052
Epoch 192/200
2207/2207 [==============================] - 0s 215us/step - loss: 0.0052
Epoch 193/200
2207/2207 [==============================] - 1s 234us/step - loss: 0.0051
Epoch 194/200
2207/2207 [==============================] - 1s 232us/step - loss: 0.0050
Epoch 195/200
2207/2207 [==============================] - 1s 247us/step - loss: 0.0050
Epoch 196/200
2207/2207 [==============================] - 1s 230us/step - loss: 0.0052
Epoch 197/200
2207/2207 [==============================] - 1s 245us/step - loss: 0.0050
Epoch 198/200
2207/2207 [=======

2255/2255 [==============================] - 1s 333us/step - loss: 0.0040
Epoch 81/200
2255/2255 [==============================] - 1s 266us/step - loss: 0.0041
Epoch 82/200
2255/2255 [==============================] - 1s 308us/step - loss: 0.0041
Epoch 83/200
2255/2255 [==============================] - 1s 295us/step - loss: 0.0040
Epoch 84/200
2255/2255 [==============================] - 1s 310us/step - loss: 0.0042
Epoch 85/200
2255/2255 [==============================] - 1s 377us/step - loss: 0.0040
Epoch 86/200
2255/2255 [==============================] - 1s 343us/step - loss: 0.0040
Epoch 87/200
2255/2255 [==============================] - 1s 322us/step - loss: 0.0041
Epoch 88/200
2255/2255 [==============================] - 1s 309us/step - loss: 0.0040
Epoch 89/200
2255/2255 [==============================] - 1s 309us/step - loss: 0.0040
Epoch 90/200
2255/2255 [==============================] - 1s 338us/step - loss: 0.0040
Epoch 91/200
2255/2255 [==============================] 

2255/2255 [==============================] - 1s 358us/step - loss: 0.0037
Epoch 174/200
2255/2255 [==============================] - 1s 340us/step - loss: 0.0036
Epoch 175/200
2255/2255 [==============================] - 1s 352us/step - loss: 0.0036
Epoch 176/200
2255/2255 [==============================] - 1s 345us/step - loss: 0.0036
Epoch 177/200
2255/2255 [==============================] - 1s 342us/step - loss: 0.0035
Epoch 178/200
2255/2255 [==============================] - 1s 397us/step - loss: 0.0036
Epoch 179/200
2255/2255 [==============================] - 1s 362us/step - loss: 0.0036
Epoch 180/200
2255/2255 [==============================] - 1s 376us/step - loss: 0.0035
Epoch 181/200
2255/2255 [==============================] - 1s 368us/step - loss: 0.0037
Epoch 182/200
2255/2255 [==============================] - 1s 357us/step - loss: 0.0037
Epoch 183/200
2255/2255 [==============================] - 1s 416us/step - loss: 0.0036
Epoch 184/200
2255/2255 [=====================

...

1849/1849 [==============================] - 1s 276us/step - loss: 0.0052
Epoch 145/200
1849/1849 [==============================] - 1s 281us/step - loss: 0.0054
Epoch 146/200
1849/1849 [==============================] - 1s 295us/step - loss: 0.0052
Epoch 147/200
1849/1849 [==============================] - 0s 254us/step - loss: 0.0055
Epoch 148/200
1849/1849 [==============================] - 1s 284us/step - loss: 0.0052
Epoch 149/200
1849/1849 [==============================] - 0s 262us/step - loss: 0.0055
Epoch 150/200
1849/1849 [==============================] - 0s 242us/step - loss: 0.0060
Epoch 151/200
1849/1849 [==============================] - 0s 231us/step - loss: 0.0056
Epoch 152/200
1849/1849 [==============================] - 1s 276us/step - loss: 0.0056
Epoch 153/200
1849/1849 [==============================] - 1s 296us/step - loss: 0.0060
Epoch 154/200
1849/1849 [==============================] - 1s 295us/step - loss: 0.0058
Epoch 155/200
1849/1849 [=====================

2111/2111 [==============================] - 1s 243us/step - loss: 0.0038
Epoch 37/200
2111/2111 [==============================] - 1s 251us/step - loss: 0.0036
Epoch 38/200
2111/2111 [==============================] - 1s 257us/step - loss: 0.0037
Epoch 39/200
2111/2111 [==============================] - 1s 258us/step - loss: 0.0038
Epoch 40/200
2111/2111 [==============================] - 1s 271us/step - loss: 0.0036
Epoch 41/200
2111/2111 [==============================] - 0s 235us/step - loss: 0.0047
Epoch 42/200
2111/2111 [==============================] - 1s 250us/step - loss: 0.0035
Epoch 43/200
2111/2111 [==============================] - 1s 250us/step - loss: 0.0041
Epoch 44/200
2111/2111 [==============================] - 1s 300us/step - loss: 0.0036
Epoch 45/200
2111/2111 [==============================] - 1s 309us/step - loss: 0.0037
Epoch 46/200
2111/2111 [==============================] - 1s 281us/step - loss: 0.0036
Epoch 47/200
2111/2111 [==============================] 

2111/2111 [==============================] - 1s 250us/step - loss: 0.0033
Epoch 130/200
2111/2111 [==============================] - 1s 269us/step - loss: 0.0038
Epoch 131/200
2111/2111 [==============================] - 1s 244us/step - loss: 0.0032
Epoch 132/200
2111/2111 [==============================] - 0s 229us/step - loss: 0.0037
Epoch 133/200
2111/2111 [==============================] - 0s 230us/step - loss: 0.0032
Epoch 134/200
2111/2111 [==============================] - 1s 252us/step - loss: 0.0035
Epoch 135/200
2111/2111 [==============================] - 1s 270us/step - loss: 0.0032
Epoch 136/200
2111/2111 [==============================] - 0s 231us/step - loss: 0.0035
Epoch 137/200
2111/2111 [==============================] - 1s 241us/step - loss: 0.0032
Epoch 138/200
2111/2111 [==============================] - 1s 248us/step - loss: 0.0036
Epoch 139/200
2111/2111 [==============================] - 1s 244us/step - loss: 0.0032
Epoch 140/200
2111/2111 [=====================

2184/2184 [==============================] - 1s 233us/step - loss: 0.0055
Epoch 22/200
2184/2184 [==============================] - 1s 275us/step - loss: 0.0057
Epoch 23/200
2184/2184 [==============================] - 1s 266us/step - loss: 0.0063
Epoch 24/200
2184/2184 [==============================] - 1s 271us/step - loss: 0.0063
Epoch 25/200
2184/2184 [==============================] - 1s 270us/step - loss: 0.0059
Epoch 26/200
2184/2184 [==============================] - 1s 261us/step - loss: 0.0058
Epoch 27/200
2184/2184 [==============================] - 1s 287us/step - loss: 0.0061
Epoch 28/200
2184/2184 [==============================] - 1s 281us/step - loss: 0.0055
Epoch 29/200
2184/2184 [==============================] - 1s 271us/step - loss: 0.0057
Epoch 30/200
2184/2184 [==============================] - 1s 279us/step - loss: 0.0056
Epoch 31/200
2184/2184 [==============================] - 1s 302us/step - loss: 0.0058
Epoch 32/200
2184/2184 [==============================] 

2184/2184 [==============================] - 1s 298us/step - loss: 0.0045
Epoch 116/200
2184/2184 [==============================] - 1s 281us/step - loss: 0.0045
Epoch 117/200
2184/2184 [==============================] - 1s 261us/step - loss: 0.0046
Epoch 118/200
2184/2184 [==============================] - 1s 277us/step - loss: 0.0045
Epoch 119/200
2184/2184 [==============================] - 1s 251us/step - loss: 0.0045
Epoch 120/200
2184/2184 [==============================] - 1s 232us/step - loss: 0.0045
Epoch 121/200
2184/2184 [==============================] - 1s 263us/step - loss: 0.0050
Epoch 122/200
2184/2184 [==============================] - 1s 234us/step - loss: 0.0045
Epoch 123/200
2184/2184 [==============================] - 1s 256us/step - loss: 0.0045
Epoch 124/200
2184/2184 [==============================] - 1s 254us/step - loss: 0.0044
Epoch 125/200
2184/2184 [==============================] - 1s 267us/step - loss: 0.0045
Epoch 126/200
2184/2184 [=====================

In [4]:
rmse_list

{'584-ws-training.csv': 27.978256882533103,
 '567-ws-training.csv': 25.567757336406896,
 '596-ws-training.csv': 21.84079979168464,
 '552-ws-training.csv': 18.067069812362195,
 '544-ws-training.csv': 18.983288734946996,
 '540-ws-training.csv': 32.12453390115133}

time: 7.15 ms


In [5]:
mae_list

{'584-ws-training.csv': 20.242002014486324,
 '567-ws-training.csv': 17.505947185926026,
 '596-ws-training.csv': 16.07470903523763,
 '552-ws-training.csv': 12.295386124816709,
 '544-ws-training.csv': 14.070348023941994,
 '540-ws-training.csv': 24.936448892393063}

time: 2.79 ms


In [6]:
from statistics import pstdev, mean

rmse_values = rmse_list.values()
mae_values = mae_list.values()

print ('Mean - RMSE : {}'.format(mean(rmse_values)))
print ('Standard Deviation - RMSE : {}'.format(pstdev(rmse_values)))

print ('Mean - MAE : {}'.format(mean(mae_values)))
print ('Standard Deviation - MAE : {}'.format(pstdev(mae_values)))

Mean - RMSE : 24.09361774318086
Standard Deviation - RMSE : 4.987802695527861
Mean - MAE : 17.520806879466956
Standard Deviation - MAE : 4.157297952378541
time: 9.61 ms
